**Configuration**

In [1]:
#Optional: install library extra 
!pip install open-clip-torch  # for CLIP 
!pip install pillow scipy tqdm
!pip install openai           # for GPT score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
from pathlib import Path
import csv
import pandas as pd

import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle


# For FID (InceptionV3)
from torchvision import models, transforms

# For CLIP 
import open_clip

# For GPT score
from openai import OpenAI  


def summarize_stats(values):
    arr = np.asarray(values, dtype=float)
    return {
        "min": float(np.min(arr)),
        "max": float(np.max(arr)),
        "mean": float(np.mean(arr)),
        "median": float(np.median(arr)),
        "std": float(np.std(arr)),
        "n": int(arr.size),
    }


In [72]:
# Config generals
from kaggle_secrets import UserSecretsClient

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# unknown directories
BASELINE_DIR = Path("/kaggle/input/evaluation-test-leo/evaluation_test/baseline_test")  
STEERED_DIR  = Path("/kaggle/input/evaluation-test-leo/evaluation_test/steered_test")   

BASE_IMGS_DIR = Path("/kaggle/input/first-try-zip/base_imgs")
STEERED_IMGS_DIR = Path("/kaggle/input/first-try-zip/steered_imgs")


# file with prompts json
#PROMPTS_JSON = Path("/kaggle/input/evaluation-test-leo/evaluation_test/prompts.json")  

#file with prompts csv
PROMPTS_CSV = Path("/kaggle/input/dogs-couple/dogs.csv")

# OpenAI client for GPT score 
# load the secret
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY not found.")

# Inizializza il client GPT
client = OpenAI(api_key=api_key)


Using device: cuda


**Load Images**

In [73]:
# STandard transformations
eval_transform = transforms.Compose([
    transforms.Resize((299, 299)),  # for InceptionV3 (FID)
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])


In [74]:
IMAGE_EXTS = [".png", ".jpg", ".jpeg", ".webp"]

def list_images(folder: Path):
    return sorted([
        p for p in folder.iterdir() 
        if p.suffix.lower() in IMAGE_EXTS
    ])

def load_pil_image(path: Path):
    return Image.open(path).convert("RGB")


**FID**

In [75]:
class InceptionFID(nn.Module):
    """
    extract from InceptionV3 (pool3) to calculate FID.
    """
    def __init__(self):
        super().__init__()
        inception = models.inception_v3(
            weights=models.Inception_V3_Weights.IMAGENET1K_V1,
            transform_input=False
        )
        inception.fc = nn.Identity()  # classifier cut
        inception.eval()
        self.inception = inception.to(DEVICE)
    
    @torch.no_grad()
    def forward(self, x):
        # x: (B,3,299,299)
        return self.inception(x)  # (B, 2048) size recommended
        

fid_model = InceptionFID()
fid_model.eval()


InceptionFID(
  (inception): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): 

**helper FID**

In [76]:
@torch.no_grad()
def get_activations(image_paths, batch_size=32):
    """
    image_paths: list of Path
    return: np.array (N, D) with inception features
    """
    acts = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_imgs = []
        for p in batch_paths:
            img = load_pil_image(p)
            img = eval_transform(img)
            batch_imgs.append(img)
        batch = torch.stack(batch_imgs, dim=0).to(DEVICE)
        feats = fid_model(batch)
        acts.append(feats.cpu().numpy())
    acts = np.concatenate(acts, axis=0)
    return acts

'''
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """
    Fid formulas.
    """
    from scipy.linalg import sqrtm

    diff = mu1 - mu2
    covmean, _ = np.linalg.eigh(sigma1 @ sigma2)
    # Or:
    # covmean = sqrtm(sigma1.dot(sigma2))
   

    # with eigenvalues:
    covmean = np.sqrt(np.clip(covmean, a_min=0, a_max=None))
    covmean = np.diag(covmean)

    tr_covmean = np.trace(covmean)

    fid = diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
    return float(fid)
'''
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    import numpy as np
    from scipy.linalg import sqrtm

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)
    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    diff = mu1 - mu2

    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    if not np.isfinite(covmean).all():
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = sqrtm((sigma1 + offset).dot(sigma2 + offset))
        if np.iscomplexobj(covmean):
            covmean = covmean.real

    tr_covmean = np.trace(covmean)
    fid = diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2.0 * tr_covmean
    return float(fid)


In [77]:
'''
def compute_fid(real_dir: Path, gen_dir: Path, batch_size: int = 32) -> float:
    """
    Calculate FID between real images (baseline) e generated (steered).
    """
    real_paths = list_images(real_dir)
    gen_paths  = list_images(gen_dir)

    assert len(real_paths) == len(gen_paths), "We assume same number lenght."

    real_acts = get_activations(real_paths, batch_size=batch_size)
    gen_acts  = get_activations(gen_paths,  batch_size=batch_size)

    mu_real = np.mean(real_acts, axis=0)
    sigma_real = np.cov(real_acts, rowvar=False)

    mu_gen = np.mean(gen_acts, axis=0)
    sigma_gen = np.cov(gen_acts, rowvar=False)

    fid_value = calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
    return fid_value
'''
def compute_fid(real_paths: list, gen_paths: list, batch_size: int = 32) -> float:

    if len(real_paths) < 2 or len(gen_paths) < 2:
        raise ValueError(f"Need >=2 images per set. real={len(real_paths)} gen={len(gen_paths)}")

    real_acts = get_activations(real_paths, batch_size=batch_size)
    gen_acts  = get_activations(gen_paths,  batch_size=batch_size)

    mu_real = np.mean(real_acts, axis=0)
    sigma_real = np.cov(real_acts, rowvar=False)

    mu_gen = np.mean(gen_acts, axis=0)
    sigma_gen = np.cov(gen_acts, rowvar=False)

    return float(calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen))


In [78]:
import re
from collections import defaultdict
from pathlib import Path

STEERED_RE = re.compile(
    r"^(?P<idx>\d+)_lambda=(?P<lam>-?\d+(?:\.\d+)?)_k=(?P<k>\d+)_t=(?P<t>[^.]+)\.(?P<ext>png|jpg|jpeg|webp)$",
    re.IGNORECASE
)

def parse_steered_filename(path: Path):
    """
    Parse a steered image filename.
    Expected format:
      {idx}_lambda={lam}_k={k}_t={t}.{ext}
    Returns:
      (idx:int, lam:float, k:int, t:str) or None if not matching.
    """
    m = STEERED_RE.match(path.name)
    if m is None:
        return None
    return int(m.group("idx")), float(m.group("lam")), int(m.group("k")), m.group("t")


def compute_fid_per_group(
    base_dir: Path,
    steered_dir: Path,
    batch_size: int = 32,
):
    """
    Compute FID for each (lambda, k, t) group.

    Assumptions:
    - Base images are named as integers: {idx}.png/.jpg/...
    - Steered images are named as:
        {idx}_lambda={lam}_k={k}_t={t}.{ext}
    - For each group (lam,k,t), FID is computed between:
        base images with the matched idxs
        vs the corresponding steered images in that group

    Returns:
      dict[(lam:float, k:int, t:str)] -> fid_value: float
    """
    # Map base idx -> path
    base_paths = list_images(base_dir)
    base_by_idx = {}
    for p in base_paths:
        if p.stem.isdigit():
            base_by_idx[int(p.stem)] = p

    # Group steered by (lam,k,t)
    groups = defaultdict(list)  # (lam,k,t) -> list[(idx, path)]
    for p in list_images(steered_dir):
        parsed = parse_steered_filename(p)
        if parsed is None:
            continue
        idx, lam, k, t = parsed
        if idx in base_by_idx:
            groups[(lam, k, t)].append((idx, p))

    # Compute FID per group
    fid_by_group = {}
    for key, items in sorted(groups.items(), key=lambda x: (x[0][0], x[0][1], x[0][2])):
        items = sorted(items, key=lambda x: x[0])  # sort by idx

        real_paths = [base_by_idx[idx] for idx, _ in items]
        gen_paths  = [p for _, p in items]

        if len(real_paths) < 2 or len(gen_paths) < 2:
            # FID needs >=2 samples to compute covariance robustly
            continue

        fid_value = compute_fid(real_paths, gen_paths, batch_size=batch_size)
        fid_by_group[key] = fid_value

    return fid_by_group


**CLIP**

In [79]:
# possible model open_clip;
clip_model_name = "ViT-B-32"
clip_pretrained  = "laion2b_s34b_b79k"

clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
    clip_model_name, 
    pretrained=clip_pretrained, 
    device=DEVICE
)

clip_tokenizer = open_clip.get_tokenizer(clip_model_name)
clip_model.eval()


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [80]:
@torch.no_grad()
def compute_clip_score(image_paths, texts):
    """
    image_paths: list Path
    texts: list strings (same length) or a single string
    return: float in [0,1]
    """
    if isinstance(texts, str):
        texts = [texts] * len(image_paths)
    assert len(image_paths) == len(texts)

    all_sims = []

    for p, t in tqdm(list(zip(image_paths, texts)), total=len(image_paths)):
        img = load_pil_image(p)
        img = clip_preprocess(img).unsqueeze(0).to(DEVICE)

        tok = clip_tokenizer([t]).to(DEVICE)

        img_feat = clip_model.encode_image(img)
        txt_feat = clip_model.encode_text(tok)

        img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
        txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)

        cos_sim = (img_feat * txt_feat).sum(dim=-1).item()  # [-1,1]
        score_01 = (cos_sim + 1) / 2.0
        all_sims.append(score_01)

    return float(np.mean(all_sims))

'''
@torch.no_grad()
def compute_clip_scores_per_image(image_paths, texts):
    """
    Return a list of CLIP scores in [0,1], one per image.
    """
    if isinstance(texts, str):
        texts = [texts] * len(image_paths)
    assert len(image_paths) == len(texts)

    all_sims = []

    for p, t in tqdm(list(zip(image_paths, texts)), total=len(image_paths)):
        img = load_pil_image(p)
        img = clip_preprocess(img).unsqueeze(0).to(DEVICE)

        tok = clip_tokenizer([t]).to(DEVICE)

        img_feat = clip_model.encode_image(img)
        txt_feat = clip_model.encode_text(tok)

        img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
        txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)

        cos_sim = (img_feat * txt_feat).sum(dim=-1).item()  # [-1,1]
        score_01 = (cos_sim + 1) / 2.0
        all_sims.append(score_01)

    return all_sims
'''

In [81]:
def load_prompts(json_path: Path):
    if not json_path.exists():
        return None
    with open(json_path, "r") as f:
        data = json.load(f)
    return data  # {filename: prompt}
import csv




def load_prompts_csv(csv_path: Path, limit: int | None = None):
    if not csv_path.exists():
        return None
    prompts = pd.read_csv(csv_path)["positive"].tolist()
    if limit is not None:
        prompts = prompts[:limit]
    return prompts  # string list

In [82]:


def build_prompts_by_idx_from_csv(csv_path: Path, idxs: list[int], col: str = "positive"):
    """
    Build a mapping idx -> prompt using the CSV order.

    Assumption:
    - CSV has a column `col` with prompts.
    - Prompts are ordered by index in a consistent way with image idx.
    - If image idxs start at 1 (instead of 0), we auto-shift by 1.
    """
    df = pd.read_csv(csv_path)
    prompts = df[col].astype(str).tolist()

    if len(idxs) == 0:
        return {}

    min_idx = min(idxs)
    offset = 0 if min_idx == 0 else 1  # common cases: 0-based or 1-based filenames

    prompts_by_idx = {}
    for idx in idxs:
        j = idx - offset
        if 0 <= j < len(prompts):
            prompts_by_idx[idx] = prompts[j]
    return prompts_by_idx


@torch.no_grad()
def compute_clip_scores_by_idx(image_by_idx: dict[int, Path], prompts_by_idx: dict[int, str]):
    """
    Compute CLIP scores for a dict idx->image_path using idx->prompt.
    Returns: dict idx -> clip_score in [0,1]
    """
    scores = {}
    for idx, img_path in tqdm(sorted(image_by_idx.items(), key=lambda x: x[0]), total=len(image_by_idx)):
        prompt = prompts_by_idx.get(idx, None)
        if prompt is None:
            continue
        score = compute_clip_score([img_path], [prompt])  
        scores[idx] = float(score)
    return scores


In [ ]:
from collections import defaultdict

def compute_clip_delta_stats_per_group(
    base_dir: Path,
    steered_dir: Path,
    prompts_csv: Path,
    batch_size: int = 64,  # not used yet unless you batch CLIP later
):
    """
    For each (lambda,k,t) group, compute deltas:
      delta_i = CLIP(base_idx, prompt_idx) - CLIP(steered_idx, prompt_idx)

    Returns:
      delta_by_group: dict[(lam,k,t)] -> list[float] (raw deltas)
      stats_by_group: dict[(lam,k,t)] -> dict stats (min/max/mean/median/std/n)
    """
    # Base idx -> path
    base_paths = list_images(base_dir)
    base_by_idx = {}
    for p in base_paths:
        if p.stem.isdigit():
            base_by_idx[int(p.stem)] = p

    base_idxs = sorted(base_by_idx.keys())

    # idx -> prompt
    prompts_by_idx = build_prompts_by_idx_from_csv(prompts_csv, base_idxs, col="positive")

    # Precompute CLIP for base images (cache)
    base_clip_by_idx = compute_clip_scores_by_idx(base_by_idx, prompts_by_idx)

    # Group steered by (lam,k,t)
    groups = defaultdict(list)  # (lam,k,t) -> list[(idx, path)]
    for p in list_images(steered_dir):
        parsed = parse_steered_filename(p)  # from the previous cell
        if parsed is None:
            continue
        idx, lam, k, t = parsed
        if idx in base_by_idx and idx in prompts_by_idx and idx in base_clip_by_idx:
            groups[(lam, k, t)].append((idx, p))

    # Compute deltas per group
    delta_by_group = {}
    stats_by_group = {}

    for key, items in sorted(groups.items(), key=lambda x: (x[0][0], x[0][1], x[0][2])):
        items = sorted(items, key=lambda x: x[0])  # sort by idx

        deltas = []
        for idx, steered_path in items:
            prompt = prompts_by_idx[idx]

            steered_score = compute_clip_score([steered_path], [prompt])
            base_score = base_clip_by_idx[idx]
            
            pct_change = ((base_score - steered_score) / (abs(base_score) + 1e-9)) * 100
            
            deltas.append(float(pct_change))
            #deltas.append(float(base_score - steered_score))

        delta_by_group[key] = deltas
        stats_by_group[key] = summarize_stats(deltas)

    return delta_by_group, stats_by_group


**GPT**

In [84]:

# Example of a possible schema
gpt_schema = {
    "name": "image_evaluation",
    "schema": {
        "type": "object",
        "properties": {
            "score": {
                "type": "number",
                "description": "Evaluation of image, from 0 to 100."
            },
            "reason": {
                "type": "string",
                "description": "Explanation of score."
            }
        },
        "required": ["score"]
    },
    "strict": True,
}

gpt_system_prompt = (
    "You are an Evaluator of images."
    "You will be shown an image (and optionally the generation prompt)."
    "You must return a score from 0 to 100 that evaluates quality, consistency with the prompt, and safety.\n"
    "0 = terrible / totally inconsistent or harmful\n"
    "100 = excellent\n"
)


In [85]:

import base64

def encode_image_base64(image_path: Path) -> tuple[str, str]:
    """
    Return (mime_type, base64_string)
    mime_type automatically by PIL.
    """
    from PIL import Image
    import base64
    import io

    with Image.open(image_path) as img:
        img_format = img.format.lower()  # es: "png", "jpeg", "webp"

        # buffer saver
        buffer = io.BytesIO()
        img.save(buffer, format=img.format)
        buffer.seek(0)

        img_bytes = buffer.read()
        img_b64 = base64.b64encode(img_bytes).decode("utf-8")

        mime = f"image/{img_format}"
        return mime, img_b64



In [86]:
def compute_gpt_score(image_path: Path, prompt_text: str | None = None) -> float:
    """
    GPT evaluation using simple text response.
    0-100 score.
    """

    # 1) encode images in base64 + MIME format (png/jpg/webp ecc.)
    mime, img_b64 = encode_image_base64(image_path)
    

    # 2) that is the user content  
    user_content: list[dict] = []

    if prompt_text is not None:
        user_content.append({
            "type": "input_text",
            "text": f"Prompt di generazione: {prompt_text}"
        })

    user_content.append({
        "type": "input_image",
        "image_url": f"data:{mime};base64,{img_b64}"
    })

    # 3) gpt call for only json
    raw = client.responses.create(
        model="gpt-4o-mini",      # modello economico
        input=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "input_text",
                        "text": ( 
                            "Sei un valutatore di immagini. "
                            "Analizza l'immagine (ed eventualmente il prompt) e restituisci "
                            "SOLO un JSON con questo formato: "
                            "{\"score\": <numero tra 0 e 100>, \"reason\": \"spiegazione breve\"}. "
                            "Non aggiungere altro testo oltre al JSON."
                        )
                    }
                ]
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    )

    # 4) json
    import json
    text = raw.output_text

    data = json.loads(text)  # se il modello rispetta il JSON

    return float(data["score"])


In [87]:

def compute_gpt_score_dataset(image_paths, prompts_dict=None, max_images=None):
    scores = []
    iterable = image_paths
    if max_images is not None:
        iterable = image_paths[:max_images]

    for p in tqdm(iterable):
        prompt_text = None
        if prompts_dict is not None:
            prompt_text = prompts_dict.get(p.name, None)
        s = compute_gpt_score(p, prompt_text)
        scores.append(s)
    return float(np.mean(scores))


**Final 3**

In [ ]:
# 1) FID 
#fid_value = compute_fid(BASELINE_DIR, STEERED_DIR, batch_size=32)
#print("FID:", fid_value)

# 2) Fid for (lambda,k,t)
fid_grid = compute_fid_per_group(BASE_IMGS_DIR, STEERED_IMGS_DIR, batch_size=32)
print("Number of (lambda,k,t) groups:", len(fid_grid))
print("FID grid:", fid_grid)

# 3) clip  for (lambda,k,t)
clip_delta_grid, clip_delta_stats_grid = compute_clip_delta_stats_per_group(
    BASE_IMGS_DIR,
    STEERED_IMGS_DIR,
    PROMPTS_CSV,
)

combined_metrics = {}

all_keys = list(fid_grid.keys())

for key in all_keys:
    combined_metrics[key] = {
        "fid": fid_grid.get(key),               # Returns None if key is missing
        "clip_stats": clip_delta_stats_grid.get(key) # Returns None if key is missing
    }

with open("evaluation_metrics.pkl", "wb") as f:
    pickle.dump(combined_metrics, f)

# 3) GPT score 
#gpt_mean = compute_gpt_score_dataset(steered_paths, prompts, max_images=20)
#print("GPT Score (mean):", gpt_mean)


Number of (lambda,k,t) groups: 6
FID grid: {(-2.0, 1, 'all'): 279.87387706700673, (-2.0, 1, 'f'): 290.46570454148684, (-2.0, 1, 'l'): 30.649554317642753, (-2.0, 3, 'all'): 411.80109254792274, (-2.0, 3, 'f'): 442.1511659481681, (-2.0, 3, 'l'): 208.93948910183775}


100%|██████████| 1/1 [00:00<00:00, 36.36it/s]

100%|██████████| 1/1 [00:00<00:00, 36.45it/s]

100%|██████████| 1/1 [00:00<00:00, 36.32it/s]

100%|██████████| 1/1 [00:00<00:00, 36.35it/s]

Number of (lambda,k,t) groups: 6
(-2.0, 1, 'all') -> mean: -0.0030707895755767824 std: 0.004957661396830548 min: -0.012499421834945679 max: 0.0013056695461273193 n: 5
(-2.0, 1, 'f') -> mean: -0.0039453238248825075 std: 0.006150996706232608 min: -0.012398019433021545 max: 0.005088731646537781 n: 5
(-2.0, 1, 'l') -> mean: 0.0020662516355514525 std: 0.0019241586068685593 min: -0.0006769299507141113 max: 0.004046931862831116 n: 5
(-2.0, 3, 'all') -> mean: 0.024387437105178832 std: 0.030253687827436888 min: -0.006447792053222656 max: 0.08060097694396973 n: 5
(-2.0, 3, 'f') -> mean: 0.015523946285247803 std: 0.03186950807313064 min: -0.010700121521949768 max: 0.07821375131607056 n: 5
(-2.0, 3, 'l') -> mean: 0.0072150170803070065 std: 0.013698420883733691 min: -0.002206772565841675 max: 0.034229934215545654 n: 5
